In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Bengaluru_House_Data.csv')

In [3]:
df.shape

(13320, 9)

In [4]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


## Lets work on the duplicate rows

In [305]:
df.duplicated().sum()

np.int64(529)

In [306]:
df = df.drop_duplicates()

In [307]:
df.shape

(12791, 9)

## Lets work on the missing values

In [308]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5328
total_sqft         0
bath              73
balcony          605
price              0
dtype: int64

In [309]:
(df.isnull().sum() * 100)/len(df)

area_type        0.000000
availability     0.000000
location         0.007818
size             0.125088
society         41.654288
total_sqft       0.000000
bath             0.570714
balcony          4.729888
price            0.000000
dtype: float64

In [310]:
## filling the missing value in location to 'other'
df = df.dropna(subset= ['location'])

In [311]:
df.shape

(12790, 9)

In [312]:
df['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', nan, '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [313]:
def extract_bhk(x):
    try:
        x = str(x).lower()
        if 'rk' in x:
            return 1
        return int(x.split()[0])
    except:
        return None

df['bhk'] = df['size'].apply(extract_bhk)

In [314]:
df['bhk'].isnull().sum()

np.int64(16)

In [315]:
## Filling the missing values in 'bhk' with median, since most of the values lies in between 1 to 3
median_bhk = df['bhk'].median()
df['bhk'] = df['bhk'].fillna(median_bhk)

In [316]:
df.isna().sum()

area_type          0
availability       0
location           0
size              16
society         5328
total_sqft         0
bath              73
balcony          605
price              0
bhk                0
dtype: int64

In [317]:
df['society'].nunique()

2688

In [318]:
df['society'].value_counts().head(10)

society
GrrvaGr    68
PrarePa    63
Sryalan    56
Prtates    54
GMown E    52
Bhmesy     51
Prarkun    48
IBityin    45
PrityTr    42
Prityel    41
Name: count, dtype: int64

In [319]:
## Since missingness is very high, don’t try to impute with median/mode
## Capture whether society info is present at all
df['has_society'] = df['society'].notnull().astype(int)

In [320]:
## capture the strength of signal from frequently occurring societies.
society_freq = df['society'].value_counts() 
df['society_freq'] = df['society'].map(society_freq) 
df['society_freq'] = df['society_freq'].fillna(0)

In [321]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price,bhk,has_society,society_freq
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07,2.0,1,5.0
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00,4.0,1,12.0
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00,3.0,0,0.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00,3.0,1,22.0
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00,2.0,0,0.0


In [322]:
## Bathrooms are usually proportional to BHK.
## You can impute missing values based on bhk (e.g., 2 BHK → 2 baths, 3 BHK → 3 baths).
df.loc[df['bath'].isnull(), 'bath'] = df.loc[df['bath'].isnull(), 'bhk']

In [323]:
df.isna().sum()

area_type          0
availability       0
location           0
size              16
society         5328
total_sqft         0
bath               0
balcony          605
price              0
bhk                0
has_society        0
society_freq       0
dtype: int64

In [324]:
## Balcony count is numeric but categorical-like (most homes have 0–2 balconies)
df['balcony'] = df['balcony'].fillna(df['balcony'].median())

In [325]:
df.isna().sum()

area_type          0
availability       0
location           0
size              16
society         5328
total_sqft         0
bath               0
balcony            0
price              0
bhk                0
has_society        0
society_freq       0
dtype: int64

In [326]:
df = df.drop( ['size', 'society'], axis = 1)

In [327]:
df.head()

,area_type,availability,location,total_sqft,bath,balcony,price,bhk,has_society,society_freq
0,Super built-up Area,19-Dec,Electronic City Phase II,1056,2.0,1.0,39.07,2.0,1,5.0
1,Plot Area,Ready To Move,Chikka Tirupathi,2600,5.0,3.0,120.00,4.0,1,12.0
2,Built-up Area,Ready To Move,Uttarahalli,1440,2.0,3.0,62.00,3.0,0,0.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521,3.0,1.0,95.00,3.0,1,22.0
4,Super built-up Area,Ready To Move,Kothanur,1200,2.0,1.0,51.00,2.0,0,0.0


In [328]:
df.isna().sum()

area_type       0
availability    0
location        0
total_sqft      0
bath            0
balcony         0
price           0
bhk             0
has_society     0
society_freq    0
dtype: int64

## 🛠️ Handling Missing Values

### 1. Location
- **Observation**: Only one missing value.  
- **Action**: Dropped the row to avoid unnecessary imputation.

### 2. Size
- **Observation**: 16 missing values; inconsistent labels ("2 BHK", "3 Bedroom", "1 RK").  
- **Action**: Extracted numeric BHK values, imputed missing with median (most homes between 1–3 BHK), then dropped the original `size` column.

### 3. Society
- **Observation**: High missing rate (40–50%).  
- **Actions**:  
  - Created `has_society` binary flag (1 if present, 0 if missing).  
  - Created `society_freq` feature (frequency of each society).  
  - Dropped the original `society` column.

### 4. Bathrooms
- **Observation**: 73 missing values; bathrooms strongly correlated with BHK.  
- **Action**: Filled missing values with the corresponding number of BHKs.

### 5. Balcony
- **Observation**: ~605 missing values (~5–6%); most homes have 0–2 balconies.  
- **Action**: Imputed missing values with the median balcony count.


In [329]:
df['availability'].unique()

array(['19-Dec', 'Ready To Move', '18-May', '18-Feb', '18-Nov', '20-Dec',
       '17-Oct', '21-Dec', '19-Sep', '20-Sep', '18-Mar', '20-Feb',
       '18-Apr', '20-Aug', '18-Oct', '19-Mar', '17-Sep', '18-Dec',
       '17-Aug', '19-Apr', '18-Jun', '22-Dec', '22-Jan', '18-Aug',
       '19-Jan', '17-Jul', '18-Jul', '21-Jun', '20-May', '19-Aug',
       '18-Sep', '17-May', '17-Jun', '21-May', '18-Jan', '20-Mar',
       '17-Dec', '16-Mar', '19-Jun', '22-Jun', '19-Jul', '21-Feb',
       'Immediate Possession', '19-May', '17-Nov', '20-Oct', '20-Jun',
       '19-Feb', '21-Oct', '21-Jan', '17-Mar', '17-Apr', '22-May',
       '19-Oct', '21-Jul', '21-Nov', '21-Mar', '16-Dec', '22-Mar',
       '20-Jan', '21-Sep', '21-Aug', '14-Nov', '19-Nov', '15-Nov',
       '16-Jul', '15-Jun', '17-Feb', '20-Nov', '20-Jul', '16-Sep',
       '15-Oct', '15-Dec', '16-Oct', '22-Nov', '15-Aug', '17-Jan',
       '16-Nov', '20-Apr', '16-Jan', '14-Jul'], dtype=object)

In [330]:
df[df['availability'] == 'Immediate Possession'].count()

area_type       16
availability    16
location        16
total_sqft      16
bath            16
balcony         16
price           16
bhk             16
has_society     16
society_freq    16
dtype: int64

In [331]:
def group_availability(x):
    if x in ['Ready To Move', 'Immediate Possession']:
        return 'Ready To Move'
    try:
        dt = pd.to_datetime(x, format='%y-%b', errors='coerce')
        if dt.month in [1,2,3]:
            return 'Q1 Possession'
        elif dt.month in [4,5,6]:
            return 'Q2 Possession'
        elif dt.month in [7,8,9]:
            return 'Q3 Possession'
        elif dt.month in [10,11,12]:
            return 'Q4 Possession'
    except:
        return 'Unknown'

df['availability_group'] = df['availability'].apply(group_availability)

In [332]:
df.head()

,area_type,availability,location,total_sqft,bath,balcony,price,bhk,has_society,society_freq,availability_group
0,Super built-up Area,19-Dec,Electronic City Phase II,1056,2.0,1.0,39.07,2.0,1,5.0,Q4 Possession
1,Plot Area,Ready To Move,Chikka Tirupathi,2600,5.0,3.0,120.00,4.0,1,12.0,Ready To Move
2,Built-up Area,Ready To Move,Uttarahalli,1440,2.0,3.0,62.00,3.0,0,0.0,Ready To Move
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521,3.0,1.0,95.00,3.0,1,22.0,Ready To Move
4,Super built-up Area,Ready To Move,Kothanur,1200,2.0,1.0,51.00,2.0,0,0.0,Ready To Move


In [333]:
df['availability_group'].value_counts()

availability_group
Ready To Move    10187
Q4 Possession      844
Q2 Possession      824
Q3 Possession      506
Q1 Possession      429
Name: count, dtype: int64

In [334]:
df = df.drop('availability', axis = 1)

In [335]:
df.head()

,area_type,location,total_sqft,bath,balcony,price,bhk,has_society,society_freq,availability_group
0,Super built-up Area,Electronic City Phase II,1056,2.0,1.0,39.07,2.0,1,5.0,Q4 Possession
1,Plot Area,Chikka Tirupathi,2600,5.0,3.0,120.00,4.0,1,12.0,Ready To Move
2,Built-up Area,Uttarahalli,1440,2.0,3.0,62.00,3.0,0,0.0,Ready To Move
3,Super built-up Area,Lingadheeranahalli,1521,3.0,1.0,95.00,3.0,1,22.0,Ready To Move
4,Super built-up Area,Kothanur,1200,2.0,1.0,51.00,2.0,0,0.0,Ready To Move


In [336]:
## working on the location column

In [337]:
## Spelling/formatting issues: "Whitefield" vs "White Field", "Electronic City" vs "ElectronicCity"
df['location'] = df['location'].str.strip().str.lower()

In [338]:
df['location'].nunique()

1283

In [340]:
threshold = 20
location_counts = df['location'].value_counts()
top_locations = location_counts[location_counts >= threshold].index.tolist()

In [341]:
df['location_grouped'] = df['location'].apply(
    lambda x: x if x in top_locations else 'Other'
)

In [343]:
loc_mean_price = df.groupby('location')['price'].mean()

In [344]:
df['location_target_enc'] = df['location'].map(loc_mean_price)

In [345]:
df.head()

,area_type,location,total_sqft,bath,balcony,price,bhk,has_society,society_freq,availability_group,location_grouped,location_target_enc
0,Super built-up Area,electronic city phase ii,1056,2.0,1.0,39.07,2.0,1,5.0,Q4 Possession,electronic city phase ii,48.683906
1,Plot Area,chikka tirupathi,2600,5.0,3.0,120.00,4.0,1,12.0,Ready To Move,Other,114.704375
2,Built-up Area,uttarahalli,1440,2.0,3.0,62.00,3.0,0,0.0,Ready To Move,uttarahalli,63.268535
3,Super built-up Area,lingadheeranahalli,1521,3.0,1.0,95.00,3.0,1,22.0,Ready To Move,lingadheeranahalli,115.348261
4,Super built-up Area,kothanur,1200,2.0,1.0,51.00,2.0,0,0.0,Ready To Move,kothanur,95.790625


In [346]:
def convert_total_sqft(x):
    """
    Convert total_sqft string to float in square feet.
    Handles:
    - ranges "1200 - 1400" → average
    - units: Sq. Meter, Sq. Yards, Acres, Perch, Guntha, Cents, etc.
    Returns float or None (later to NaN)
    """
    if isinstance(x, float):  # already numeric
        return x
    
    x = str(x).strip()
    # Case 1: range like "1133 - 1384"
    if '-' in x:
        tokens = x.split('-')
        if len(tokens) == 2:
            try:
                return (float(tokens[0]) + float(tokens[1])) / 2
            except:
                return None
    
    # Case 2: single number with unit
    # Remove commas if any
    x = x.replace(',', '')
    
    # Common unit conversions (approximate standard factors)
    conversion_factors = {
        'sq. meter': 10.7639,       # 1 sq.m = 10.7639 sqft
        'sq. meter': 10.7639,
        'square meter': 10.7639,
        'sq.meter': 10.7639,
        'sq meter': 10.7639,
        
        'sq. yard': 9.0,            # 1 sq.yd = 9 sqft
        'sq.yard': 9.0,
        'square yard': 9.0,    
        'sq yard': 9.0,
        
        'acre': 43560,              # 1 acre = 43560 sqft
        'acres': 43560,
        
        'perch': 272.25,            # 1 perch ≈ 272.25 sqft
        'guntha': 1089,             # 1 guntha = 1089 sqft
        'cents': 435.6,             # 1 cent = 435.6 sqft
        'ground': 2400,             # 1 ground ≈ 2400 sqft (South India)
    }
    
    for unit, factor in conversion_factors.items():
        if unit in x.lower():
            num_part = x.lower().replace(unit, '').strip()
            try:
                return float(num_part) * factor
            except:
                return None
    
    # Plain number (most common case)
    try:   
        return float(x)
    except:
        return None

# Apply conversion
df['total_sqft_num'] = df['total_sqft'].apply(convert_total_sqft)

# Drop original and rows where conversion failed
df = df.dropna(subset=['total_sqft_num'])
df = df.drop(columns=['total_sqft'])
df = df.rename(columns={'total_sqft_num': 'total_sqft'})

print("After total_sqft cleaning:", df.shape)        

After total_sqft cleaning: (12766, 12)


In [347]:
df.head()

,area_type,location,bath,balcony,price,bhk,has_society,society_freq,availability_group,location_grouped,location_target_enc,total_sqft
0,Super built-up Area,electronic city phase ii,2.0,1.0,39.07,2.0,1,5.0,Q4 Possession,electronic city phase ii,48.683906,1056.0
1,Plot Area,chikka tirupathi,5.0,3.0,120.00,4.0,1,12.0,Ready To Move,Other,114.704375,2600.0
2,Built-up Area,uttarahalli,2.0,3.0,62.00,3.0,0,0.0,Ready To Move,uttarahalli,63.268535,1440.0
3,Super built-up Area,lingadheeranahalli,3.0,1.0,95.00,3.0,1,22.0,Ready To Move,lingadheeranahalli,115.348261,1521.0
4,Super built-up Area,kothanur,2.0,1.0,51.00,2.0,0,0.0,Ready To Move,kothanur,95.790625,1200.0


In [220]:
df.dtypes

area_type              object
location               object
bath                  float64
balcony               float64
price                 float64
bhk                   float64
has_society             int64
society_freq          float64
availability_group     object
total_sqft            float64
dtype: object

In [348]:
df.head()

,area_type,location,bath,balcony,price,bhk,has_society,society_freq,availability_group,location_grouped,location_target_enc,total_sqft
0,Super built-up Area,electronic city phase ii,2.0,1.0,39.07,2.0,1,5.0,Q4 Possession,electronic city phase ii,48.683906,1056.0
1,Plot Area,chikka tirupathi,5.0,3.0,120.00,4.0,1,12.0,Ready To Move,Other,114.704375,2600.0
2,Built-up Area,uttarahalli,2.0,3.0,62.00,3.0,0,0.0,Ready To Move,uttarahalli,63.268535,1440.0
3,Super built-up Area,lingadheeranahalli,3.0,1.0,95.00,3.0,1,22.0,Ready To Move,lingadheeranahalli,115.348261,1521.0
4,Super built-up Area,kothanur,2.0,1.0,51.00,2.0,0,0.0,Ready To Move,kothanur,95.790625,1200.0


In [349]:
df = df.drop('location', axis = 1)

In [350]:
df.head()

,area_type,bath,balcony,price,bhk,has_society,society_freq,availability_group,location_grouped,location_target_enc,total_sqft
0,Super built-up Area,2.0,1.0,39.07,2.0,1,5.0,Q4 Possession,electronic city phase ii,48.683906,1056.0
1,Plot Area,5.0,3.0,120.00,4.0,1,12.0,Ready To Move,Other,114.704375,2600.0
2,Built-up Area,2.0,3.0,62.00,3.0,0,0.0,Ready To Move,uttarahalli,63.268535,1440.0
3,Super built-up Area,3.0,1.0,95.00,3.0,1,22.0,Ready To Move,lingadheeranahalli,115.348261,1521.0
4,Super built-up Area,2.0,1.0,51.00,2.0,0,0.0,Ready To Move,kothanur,95.790625,1200.0


In [351]:
df.shape

(12766, 11)

In [352]:
df.to_csv('preprocessed_and_featured_data.csv', index = False)

In [353]:
df.columns

Index(['area_type', 'bath', 'balcony', 'price', 'bhk', 'has_society',
       'society_freq', 'availability_group', 'location_grouped',
       'location_target_enc', 'total_sqft'],
      dtype='object')